# Версия трека V 0.1
# Быстрый способ начать работать с линейной регрессией
Почему бы не начать знакомиться с методами регрессионного анализа с винишка.

## Чтобы что?
- Попробовать метод на реальных данных
- Сформировать понимание, что вместо показателя "Качество белого вина" может быть любой признка из вашей рабочей области. Количество продаж, качества изделия, прогулы сотрудников - что угодно.

In [22]:
import pandas as pd
import requests
import statsmodels.api as sm
import io

In [4]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv'

In [17]:
def load_data(url):
    raw_data = requests.get(url).content
    frame = pd.read_csv(io.StringIO(raw_data.decode('utf-8')), delimiter=';')
    return frame

In [18]:
frame = load_data(url)

## Экспресс анализ влияния параметров белого вина на субъективную оценку экспертов

In [45]:
# посмотрим на набор данных
# quality - оценка экспертов
# остальное - химические параметры напитка
frame.head(3)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6


### Теперь проверим как влияет сахар на оценку экспертов

In [57]:
X = frame[['residual sugar']]
y = frame['quality']

In [58]:
#X = sm.add_constant(X) # adding a constant

model = sm.OLS(y, X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

                                 OLS Regression Results                                
Dep. Variable:                quality   R-squared (uncentered):                   0.586
Model:                            OLS   Adj. R-squared (uncentered):              0.586
Method:                 Least Squares   F-statistic:                              6934.
Date:                Sat, 07 Mar 2020   Prob (F-statistic):                        0.00
Time:                        10:39:10   Log-Likelihood:                         -13520.
No. Observations:                4898   AIC:                                  2.704e+04
Df Residuals:                    4897   BIC:                                  2.705e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                     coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------

В данном случаем мы не будем углубляться в анализ остатков, ошибку модели, выбросы, нормальность распределения параметров. Сейчас цель освоить базовый инструмент и далее идти по сложному пути.

### Разберем параметры модели:
    - R-squared = 0.58, объясненная дисперсия. Чем ближе к единице, тем лучше наша математическая модель приближает моделируемый объект. В данном случае оценку экспертов.
    - coef 0.5578 при параметре residual sugar: на каждую еденицу повышения сахара оценка экспертов повышается на 0.5578
    - P>|t| это p-value. В данном случае возьмем за константу, что если параметр выше 0.05, значит этот параметр не влияет на изменение целевой переменной и использовать его в модели нельзя.

### Пример два
Добавим в модель fixed acidity и проведем повторное обучение модели

In [63]:
X = frame[['residual sugar','fixed acidity']]
y = frame['quality']

In [64]:
model = sm.OLS(y, X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

                                 OLS Regression Results                                
Dep. Variable:                quality   R-squared (uncentered):                   0.959
Model:                            OLS   Adj. R-squared (uncentered):              0.959
Method:                 Least Squares   F-statistic:                          5.751e+04
Date:                Sat, 07 Mar 2020   Prob (F-statistic):                        0.00
Time:                        10:55:07   Log-Likelihood:                         -7847.5
No. Observations:                4898   AIC:                                  1.570e+04
Df Residuals:                    4896   BIC:                                  1.571e+04
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                     coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------

### Разберем параметры модели:
Добавили fixed acidity - получили неожиданный для нас результат. 
- При наличии параметра fixed acidity, residual sugar становится не значимым P>|t| > 0.05
- Если обратить внимание на распределение то левый ховст 0.025 соответсвует -0.011, верхняя же граница 0.975 соответсвтует 0.002. То есть может принимать как отрицательные, так и положительное значение.

Что произошло?
- Добавление признака изменило модель. 

Что делать?
- Т.к. мы сейчас разбираемся с регрессией на бытовом уровне дальше необходимо вручную перебрать признаки и оставить те, которые вносят вклад в модель.

## Самостоятельная работа
С помощью метода backword elimination( добавляем все признаки в модель, затем последовательно исключаем те, параметры которых нас не устраивают).

Есть библиотеки на выбор признаков, но, пока вручную. После сами разберетесь и нагруглите.

Шаг 1 - добавляем все признаки. Смотрим итог работы модели.
Шаг 2 - на основе summary модели исключаем один признак и снова повторяем шаг 1.


In [69]:
# Как вывести все назвния колонок?
frame.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')